In [ ]:
import matplotlib.image as mpimg 
import cv2
import numpy as np
import pandas as pd
import os
from keras_facenet import FaceNet

In [ ]:
embedder = FaceNet()

In [ ]:
dataset = "../data/105_persons"

In [ ]:
def get_dataset(dtype="train"):
    images = []
    targets = []
    with open(os.path.join(dataset, dtype + '.csv'), 'r') as file:
        for line in file.readlines():
            path, cls = map(lambda x: x.strip(), line.split(','))
            
            images.append(cv2.resize(mpimg.imread(path), (160, 160)))
            targets.append(int(cls))
    
    # Convert to numpy
    X = np.array(images)
    y = np.array(targets)
    
    return X, y

def get_label():
    labels = {}
    
    with open(os.path.join(dataset, 'label.txt')) as file:
        for i, person in enumerate(file.readlines()):
            labels[int(i)] = person.strip()
            
    return labels

In [ ]:
X_train, y_train = get_dataset(dtype="train")
X_test, y_test = get_dataset(dtype="test")
labels = get_label()

In [ ]:
print('X_Train data shape=', X_train.shape)
print('X_Test data shape=', X_test.shape)
print('y_Train data shape=', y_train.shape)
print('y_Test data shape=', y_test.shape)

In [ ]:
# Use the pretrained facenet for face embedding
X_train = embedder.embeddings(X_train)
X_test = embedder.embeddings(X_test)
print('Train embed shape=', X_train.shape)
print('Test embed shape=', X_test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train,y_train)

In [ ]:
model.predict(X_test)

In [ ]:
y_test

In [ ]:
model.score(X_test,y_test)

In [ ]:
def predict(file, with_label=True):
  x = mpimg.imread(file)
  x = cv2.resize(x, (160, 160))
  x = x.reshape(1, 160, 160, 3)
  x = embedder.embeddings(x)
  if(max(model.predict_proba(x)[0]) < 0.25):         #0.25 is the threshold
    print("Face not found !")
    return
  else:
    y_s = model.predict(x)
    return [labels[y] for y in y_s] if with_label else y_s

In [1]:
predict(os.path.join("./105_classes_pins_dataset/ELIZABETH_OLSEN/elizabeth_olsen0_1112.jpg"), with_label=True)

NameError: name 'predict' is not defined